# Load Modules

In [ ]:
from __future__ import print_function

# Ipynb Widgets
from  ipywidgets  import interact, interactive, fixed, interact_manual, VBox
import ipywidgets as widgets
from ipywidgets import Checkbox, interactive, SelectMultiple
from IPython.display import display

# Plotting
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as tkr
import matplotlib.dates as dates
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from plotly.widgets import GraphWidget
from plotly import tools
tools.set_credentials_file(username = 'jbolorinos', api_key='21KxVCwwiwGbTUJkXA2e')
init_notebook_mode(connected = False)

# Data Prep
import numpy as np
import pandas as pd
from datetime import datetime as dt
from datetime import timedelta

# Utilities
import warnings
import os 
import math
import sys

# CR2C
import cr2c_labdata as pld
import cr2c_hmidata as hmi
import cr2c_fielddata as fld
import cr2c_validation as val
cr2c_lr = pld.labrun() 


# Update with latest Lab Data

In [4]:
# Lab Data
cr2c_lr.process_data()

# Field Data
fld.process_data()

/anaconda3/envs/cr2c-monitoring/lib/python3.6/site-packages/pandas/core/reshape/merge.py:544: UserWarning:

merging between different levels can give an unintended result (2 levels on the left, 1 on the right)

/anaconda3/envs/cr2c-monitoring/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning:


Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike



In [131]:
hmi_data = hmi.get_data(['PIT700'],['PRESSURE'],[1],['MINUTE'])
# Group to hourly data
hmi_data.loc[:,'Time'] = hmi_data['Time'].dt.weekday_name
hmi_data = hmi_data.groupby('Time').mean()
hmi_data.reset_index(inplace = True)
hmi_data.loc[:,'Time'] = hmi_data['Time'].astype('category')
hmi_data['Time'].cat.reorder_categories(['Sunday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday'])
print(hmi_data.head(10), hmi_data.tail(10))


        Time         Year     Month     PIT700
0     Friday  2017.476923  6.507692  14.922090
1     Monday  2017.492308  6.400000  14.906753
2   Saturday  2017.476923  6.523077  14.910665
3     Sunday  2017.476923  6.569231  14.905812
4   Thursday  2017.476923  6.461538  14.893244
5    Tuesday  2017.484375  6.421875  14.918078
6  Wednesday  2017.476923  6.415385  14.907409
        Time         Year     Month     PIT700
0     Friday  2017.476923  6.507692  14.922090
1     Monday  2017.492308  6.400000  14.906753
2   Saturday  2017.476923  6.523077  14.910665
3     Sunday  2017.476923  6.569231  14.905812
4   Thursday  2017.476923  6.461538  14.893244
5    Tuesday  2017.484375  6.421875  14.918078
6  Wednesday  2017.476923  6.415385  14.907409         Time         Year     Month     PIT700
0     Friday  2017.476923  6.507692  14.922090
1     Monday  2017.492308  6.400000  14.906753
2   Saturday  2017.476923  6.523077  14.910665
3     Sunday  2017.476923  6.569231  14.905812
4   Thursday 

# Get Data and Plot

In [136]:
gw = GraphWidget()

class cr2cPlots:
    
    def __init__(self):
        
        # Initialize data classes
        self.dclasses = ['Lab Results','HMI Data','Validation Data']
        # Initialize lab data types
        self.ltypes = ['PH','COD','TSS_VSS','ALKALINITY','VFA','GasComp','Ammonia','Sulfate','TKN','BOD']
        # Initialize HMI data types
        self.hmi_types = ['WATER','GAS','MEMBRANES','TMP','PRESSURE','PH','TEMP','DPI','LEVEL']
        # Initialize validation data types
        self.val_types = ['COD Balance','Process Parameters','Instrument Validation']
        
        # Initialize data class, data type and variable type
        self.dclass = 'Lab Results'
        self.dtype = 'PH'
        self.vtype = 'Stage'
        # Initialize Dictionary of the UI State and data selections (populate with initial values)
        self.UIState = {'Lab Results':['PH',{'PH':['',{'Stage':[]}]}]}
        self.UIState['HMI Data'] = ['WATER',{'WATER':['',{'Time Period':['Hour']}]}]
        self.UIState['Validation'] = ['COD Balance',{'COD Balance':['',{'Validation':[]}]}]
        
        # Initialize data selection variables
        self.stage_sub, self.type_sub, self.tperiod, self.elids = None, None, None, None
        
        # Dictionary of lab data types and y-axis labels
        self.yaxisLabels =  {
            'COD' : 'COD (mg/L)',
            'BOD' : 'BOD (mg/L)',
            'OD' : 'OD (mg/L)',
            'TSS_VSS' : 'Suspended Solids (mg/L)',
            'PH' : 'pH',
            'ALKALINITY' : '$\\text{Alkalinity (mg/L as } CaCO_3 \\text{)}$',
            'VFA' : 'VFAs as mgCOD/L',
            'AMMONIA' : '$NH_3\\text{ (mg/L as N)}$',
            'TKN' : 'mgTKN/L',
            'SULFATE' : '$\\text{mg/L }SO_4$',
            'GASCOMP' : 'Biogas %',
            'WATER' : 'Flow (Gal)',
            'GAS': 'Biogas Production (liters)',
            'TMP': 'Trans-Membrane Pressure (psia)',
            'PRESSURE': 'Pressure (psig)',
            'TEMP': 'Temperature (°C)',
            'DPI': 'Differential Pressure (psia)',
            'LEVEL': 'Water Level (in.)'
        }
        
        # Load Lab Data
        self.labdatAll = pld.get_data(self.ltypes)
        # Initialize plot URL
        self.plot_url = 'https://plot.ly/organize/jbolorinos'
        # Initialize ntraces variable
        self.ntraces = None
        # Initialize tab_nest variable
        self.tab_master = None

    # Helper function for creating selection widgets for a type of lab data
    def create_mselect(self, dclass, dtype, vtype, select = False):
        
        # Insert into dictionary (or create if entry doesn't exist yet)
        if dtype in self.UIState[dclass][1] and vtype not in self.UIState[dclass][1][dtype][1]:
            
            self.UIState[dclass][1][dtype][1][vtype] = []
            
        elif dtype not in self.UIState[dclass][1]:
            
            self.UIState[dclass][1][dtype] = ['',{vtype:[]}]
            
        checkboxes = []
        
        # dtype is one of the lab data types 
        if dclass == 'Lab Results':
            
            labdat = self.labdatAll[dtype]
            lvals = list(labdat[vtype].unique())    
            # Add a checkbox for each value for which there are data
            for lval in lvals:
                
                if lval:
                    
                    cb = widgets.Checkbox(description = lval, value = select)
                    # Observe checkbox (so interactive output sent to self.response method)
                    cb.observe(self.response, names = 'value')
                    checkboxes.append(cb)               
                
        # dtype is one of the hmi data types    
        elif dclass == 'HMI Data':
            
            hmi_tables = hmi.get_table_names()
            
            if dtype == 'MEMBRANES':
                
                for reactor in ['Research AFMBR','Duty AFMBR']:
                    cb = widgets.Checkbox(description = reactor, value = select)
                    # Observe checkbox (so interactive output sent to self.response method)
                    cb.observe(self.response, names = 'value')
                    checkboxes.append(cb)  
                    
            else:
                
                elids = [table_name.split('_')[1] for table_name in hmi_tables if table_name.split('_')[0] == dtype]
                elids = list(set(elids))
                
                # Add a checkbox for each value for which there are data
                for elid in elids:
                    
                    cb = widgets.Checkbox(description = elid, value = select)
                    # Observe checkbox (so interactive output sent to self.response method)
                    cb.observe(self.response, names = 'value')
                    checkboxes.append(cb)  
                
        
        # dtype is one of the validation data types 
        elif dclass == 'Validation':
            
            vtype = 'Validation'
            # Insert into dictionary (or create if entry doesn't exist yet)
            if dtype in self.UIState[dclass][1]:
                
                self.UIState[dclass][1][dtype][1][vtype] = []
                
            else:
                
                self.UIState[dclass][1][dtype][1] = {vtype:[]}
                
            cb = widgets.Checkbox(description = 'Validation', value = False)
            # Observe checkbox (so interactive output sent to self.response method)
            cb.observe(self.response, names = 'value')
            checkboxes.append(cb)
        
        return checkboxes
    
    
    # Helper function for creating accordions for each type of lab data
    def create_accordion(self, dclass, dtype):
        
        
        if dclass == 'Lab Results':
            
            cbTypes  = widgets.VBox(children = self.create_mselect(dclass, dtype,'Type'))
            cbStages = widgets.VBox(children = self.create_mselect(dclass, dtype,'Stage'))
            
            if dtype == 'GasComp':
                
                accordion = widgets.Accordion(children = [cbTypes])
                accordion.set_title(0,'Type')
                
            elif dtype in ['PH','ALKALINITY','TKN','Ammonia','Sulfate']:
                
                accordion = widgets.Accordion(children = [cbStages])
                accordion.set_title(0,'Stage')
                
            else:
                
                accordion = widgets.Accordion(children = [cbTypes, cbStages])
                accordion.set_title(0,'Type')
                accordion.set_title(1,'Stage')
               
        elif dclass == 'HMI Data':

            self.UIState['HMI Data'][1][dtype] = ['',{'Time Period':'Hour'}]
            
            if dtype == 'MEMBRANES':
                
                rbTPeriods = widgets.RadioButtons(
                    options = ['Minute','Hour','Day','Week','Month','By Hour','By Weekday','By Month']
                )
                
            else:
                
                rbTPeriods = widgets.RadioButtons(
                    options = ['Hour','Day','Week','Month','By Hour','By Weekday','By Month']
                )
                
            rbTPeriods.observe(self.response, names = 'value')
                
            cbElids = widgets.VBox(children = self.create_mselect(dclass, dtype,'Element ID'))
            accordion = widgets.Accordion(children = [rbTPeriods, cbElids])
            accordion.set_title(0,'Time Period')
            
            if dtype == 'MEMBRANES':
                
                accordion.set_title(1, 'Reactor')
                
            else:
                
                accordion.set_title(1, 'Element ID')
        
        elif dclass == 'Validation':
            
            cbVals = widgets.VBox(children = self.create_mselect(dclass, dtype,'Validation'))
            accordion = widgets.Accordion(children = [cbVals])
            accordion.set_title(0, 'Validation')
        
        accordion.selected_index = None
        # Observe accordion (so interactive output sent to self.response method)
        accordion.observe(self.response, names = 'selected_index')
        
        return accordion 
    
    def create_tab(self, dclass):
        
        nest = widgets.Tab()
        accordions = []
        
        # Set dtypes variable according to dclass
        if dclass == 'Lab Results':
            
            dtypes = self.ltypes
            
        elif dclass == 'HMI Data':
            
            dtypes = self.hmi_types
            
        else:
            
            dtypes = self.val_types
            
        # Create accordions for the given dclass   
        for type_no, dtype in enumerate(dtypes):
            
            accordion = self.create_accordion(dclass, dtype)
            accordions.append(accordion)
            nest.set_title(type_no, dtype)
            
        nest.children = accordions
        nest.observe(self.response, names = 'selected_index')
        
        return nest
    
    
    def get_mselect(self):
        
        # Get the index value corresponding to the tab name
        dclassInd = [ind for ind,title in self.tab_master._titles.items() if title == self.dclass]
        dclassInd = int(dclassInd[0])
        tab = self.tab_master.children[int(dclassInd)]
        # Get the index value corresponding to the subTab name 
        dtypeInd = [ind for ind,title in tab._titles.items() if title == self.dtype][0]
        dtypeInd = int(dtypeInd[0])
        subTab = tab.children[dtypeInd]
        # Get the index corresponding to the accordion name
        vtypeInd = [ind for ind,title in subTab._titles.items() if title == self.vtype][0]
        vtypeInd = int(vtypeInd[0])
        accordion = subTab.children[vtypeInd]

        self.mselect = accordion.children

    
    def onSelectAll(self, b):
        
        if self.tab_master:
            
            self.get_mselect()
            
            for cb in self.mselect:
                
                cb.value = True
        
        
    def onClearAll(self, b):
        
        if self.tab_master:
            
            self.get_mselect()
            
            for cb in self.mselect:
                
                cb.value = False
      
    
    def create_tabNests(self):
        
        # Create global "Select All" and "Clear All" buttons
        self.selectAll = widgets.Button(description = 'Select All')
        self.selectAll.on_click(self.onSelectAll)
        self.clearAll = widgets.Button(description = 'Clear All')
        self.clearAll.on_click(self.onClearAll)

        # Create master tab
        self.tab_master = widgets.Tab()
        self.tab_master.children = \
            [self.create_tab('Lab Results'), self.create_tab('HMI Data'), self.create_tab('Validation')]
        self.tab_master.set_title(0,'Lab Results')
        self.tab_master.set_title(1, 'HMI Data')
        self.tab_master.set_title(2, 'Validation Data')
        self.tab_master.observe(self.response, names = 'selected_index')
        
        # Render!
        display(self.selectAll)
        display(self.clearAll) 
        display(self.tab_master)
    
    
    # Plotly plotting function
    def render_plot(
        self, 
        xseries, xseriesType, 
        yseries, 
        seriesName, seriesIndex, 
        ylabel, 
        plotType, plotMode, 
        xlabel = 'Date'
    ):
                
        if plotType == 'scatter':
            
            gObject = go.Scatter(x = xseries, y = yseries, mode = plotMode, name = seriesName)
            
        elif plotType == 'bar':
            
            gObject = go.Bar(x = xseries, y = yseries, opacity = 0.8, name = seriesName)
        
        gw.add_traces(gObject, new_indices = seriesIndex)
        
        if len(xseries) > 0:
            
            xrangeList = \
                [np.datetime64('2017-05-10'), dt.today()]
        
        gw.relayout(
            {'lines': 'overlay', 
                'xaxis': {
                    'title': xlabel,
                    'ticks': 'inside',
                    'range' : xrangeList,
                    'rangeselector': {
                        'buttons': list([
                            dict(
                                count = 7,
                                label = '1w',
                                step = 'day',
                                stepmode = 'backward'
                            ),
                            dict(
                                count = 1,
                                label = '1m',
                                step = 'month',
                                stepmode = 'backward'
                            ),
                            dict(
                                count = 6,
                                label = '6m',
                                step = 'month',
                                stepmode = 'backward'
                            ),
                            dict(step = 'all')
                        ])
                    },
                    'rangeslider' : {'range' : xrangeList},
                    'type' : xseriesType
                },
                'yaxis': {'title': ylabel}
            }
        )
        
        
    # Lab data plotting function
    def get_lab_plots(
        self,
        dtype,
        type_sub,
        stage_sub
    ):
        
        # Load lab data
        labdat = self.labdatAll[dtype]
        # Set the x-axis range to an empty list
        xrangeList = []

        # Clean case of lplot_list 
        dtype = dtype.upper()
        # Get y-axis label
        ylabel = self.yaxisLabels[dtype]

        if dtype == 'BOD':

            # If BOD, convert to wide
            labdat.loc[:,'Range'] = np.array([string.split(': ')[1] for string in labdat['Type'].values])
            labdat.loc[:,'Type']  = np.array([string.split(': ')[0] for string in labdat['Type'].values])
            labdat = labdat[['Date_Time','Stage','Type','Range','Value']]
            labdat.drop_duplicates(['Date_Time','Stage','Type','Range'], inplace = True)
            labdat.set_index(['Date_Time','Stage','Type','Range'], inplace = True)
            labdat = labdat.unstack('Range')
            # Get the error bar (symmetric)
            labdat['yerr'] = (labdat['Value']['Max Value'] - labdat['Value']['Min Value'])/2
            labdat.reset_index(inplace = True)
            labdat.columns = ['Date_Time','Stage','Type','Mean','Min','Max','yerr']
            type_list = labdat['Type'].unique()

        if dtype == 'OD':

            # Make sure type_sub types are in the dataset!
            if type_sub:
                
                type_list = [type_el for type_el in type_list if type_el in type_sub]
                
            else:
                
                type_list = \
                    ['Total','Soluble','Particulate'] + \
                    sorted(
                        list(
                            filter(
                                lambda x: x not in ['Total','Soluble','Particulate'], 
                                type_list
                            )
                        )
                    ) 

        # Filter to stages and types being subset to
        groupVars = ['Date_Time']
        
        if stage_sub:
            
            labdat = labdat.loc[labdat.Stage.isin(stage_sub)]
            groupVars.append('Stage')
            
        else:
            
            stage_sub = [None]
            
        if type_sub:
            
            labdat = labdat.loc[labdat.Type.isin(type_sub)]
            groupVars.append('Type')
            
        else:
            
            type_sub = [None]

        # Average all observations (by type and stage) taken on a day
        labdatChart = labdat.groupby(groupVars).mean()
        # Remove index!
        labdatChart.reset_index(inplace = True)

        # Delete all traces from graph widget 
        # (if a graph has been produced or everything has been de-selected)
        if self.ntraces or labdatChart.empty:
            gw.delete_traces(list(range(self.ntraces)))

        for sind,svar in enumerate(stage_sub):
            
            for tind,tvar in enumerate(type_sub):
                
                if not svar and not tvar:
                    
                    return
                
                elif not tvar:
                    
                    labdatSeries = labdatChart.loc[labdatChart['Stage'] == svar]
                    seriesName = svar
                    
                elif not svar and self.dtype != 'PH':
                    
                    labdatSeries = labdatChart.loc[labdatChart['Type'] == tvar]
                    seriesName = tvar
                    
                else:
                    
                    labdatSeries = \
                        labdatChart.loc[(labdatChart['Type'] == tvar) & (labdatChart['Stage'] == svar),:]
                    seriesName = tvar + ': ' + svar
                    
                seriesIndex = sind*len(type_sub) + tind
                labdatSeries.reset_index(inplace = True)
                
                self.render_plot(
                    xseries = labdatSeries['Date_Time'], 
                    xseriesType = 'date',
                    yseries = labdatSeries['Value'],
                    seriesName = seriesName, 
                    seriesIndex = seriesIndex, 
                    ylabel = ylabel, 
                    plotType = 'scatter',
                    plotMore = 'lines+markers'
                )

        # Save variable with number of traces (to delete them all before plotting other series)
        self.ntraces = len(type_sub)*len(stage_sub)
    
    
    def get_hmi_plots(self, dtype, elids, tperiod):
        
        ylabel = self.yaxisLabels[dtype]
        xseriesType = 'date'
        plotType = 'scatter'
        plotMode = 'lines'
        
        if tperiod == 'Hour':
            
            plotType = 'scatter'
        
        else:
            plotType = 'bar'
        
        # Delete all traces from graph widget 
        # (if a graph has been produced or everything has been de-selected)
        if self.ntraces:
            gw.delete_traces(list(range(self.ntraces)))
        
        if elids:
            
            for elind, elid in enumerate(elids):

                # Retrieve data
                try: # Try querying hourly data
                    
                    hmi_data = hmi.get_data([elid], [dtype], [1], ['HOUR'])
                    
                except: # Otherwise only available as minute data
                    
                    # Load minute data
                    hmi_data = hmi.get_data([elid],[dtype],[1],['MINUTE'])
                    # Group to hourly data
                    hmi_data.loc[:,'Time'] = hmi_data['Time'].values.astype('datetime64[h]')
                    hmi_data = hmi_data.groupby('Time').mean()
                    hmi_data.reset_index(inplace = True)

                if dtype in ['GAS','WATER']:
                    
                    hmi_data.loc[:,elid] = hmi_data[elid]*60

                if tperiod == 'Day':
                    
                    hmi_data.loc[:,'Time'] = hmi_data['Time'].dt.date
                    hmi_data = hmi_data.groupby('Time').mean()
                    hmi_data.reset_index(inplace = True)
                    
                    # If a flow, get the total (otherwise stay with average)
                    if dtype in ['GAS','WATER']:
                        
                        hmi_data.loc[:,elid] = hmi_data[elid]*24
                        
                elif tperiod == 'Week':
                    
                    hmi_data.loc[:,'week'] = hmi_data['Time'].dt.week
                    hmi_data.loc[:,'year'] = hmi_data['Time'].dt.year
                    hmi_data = hmi_data.groupby(['year','week']).mean()
                    hmi_data.reset_index(inplace = True)
                    hmi_data.loc[:,'month'] = 1
                    hmi_data.loc[:,'day'] = 1
                    hmi_data.loc[:,'Time'] = pd.to_datetime(pd.DataFrame(hmi_data[['year','month','day']]))
                    hmi_data.loc[:,'week'] = pd.to_timedelta(hmi_data['week']*7,unit = 'd')
                    hmi_data.loc[:,'Time'] = hmi_data['Time'] + hmi_data['week']
                    
                    # If a flow, get the total (otherwise stay with average)
                    if dtype in ['GAS','WATER']:
                        
                        hmi_data.loc[:,elid] = hmi_data[elid]*24*7
                        
                elif tperiod == 'Month':
                    
                    hmi_data.loc[:,'month'] = hmi_data['Time'].dt.month
                    hmi_data.loc[:,'year'] = hmi_data['Time'].dt.year 
                    hmi_data = hmi_data.groupby(['year','month']).mean()
                    hmi_data.reset_index(inplace = True)    
                    hmi_data.loc[:,'day'] = 1
                    hmi_data.loc[:,'Time'] = pd.to_datetime(pd.DataFrame(hmi_data[['year','month','day']]))
                    hmi_data.loc[:,'Days in Month'] = hmi_data['Time'].dt.daysinmonth
                    
                    # If a flow, get the total (otherwise stay with average)
                    if dtype in ['GAS','WATER']:
                        
                        hmi_data.loc[:,elid] = hmi_data[elid]*24*hmi_data.loc[:,'Days in Month']
                                        
                elif tperiod == 'By Hour':
                
                    hmi_data.loc[:,'Time'] = hmi_data['Time'].dt.hour
                    hmi_data = hmi_data.groupby('Time').mean()
                    hmi_data.reset_index(inplace = True)
                    xseriesType = 'linear'
                    xlabel = 'Hour of day'
                
                elif tperiod == 'By Weekday':
                    
                    hmi_data.loc[:,'Time'] = hmi_data['Time'].dt.weekday
                    hmi_data = hmi_data.groupby('Time').mean()
                    hmi_data.reset_index(inplace = True)
#                     hmi_data.loc[:,'Time'] = hmi_data['Time'].astype('category')
#                     hmi_data['Time'].cat.reorder_categories(
#                         ['Sunday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday'],
#                         inplace = True
#                     )
                    xseriesType = 'linear'
                    xlabel = 'Weekday'
                
                elif tperiod == 'By Month':
                    
                    hmi_data.loc[:,'Time'] = hmi_data['Time'].dt.month
                    hmi_data = hmi_data.groupby('Time').mean()
                    hmi_data.reset_index(inplace = True)
                    xseriesType = 'linear'
                    xlabel = 'Month'
    

                # Plot Result
                self.render_plot(
                    xseries = hmi_data['Time'],
                    xseriesType = xseriesType,
                    yseries = hmi_data[elid],
                    seriesName = elid,
                    seriesIndex = elind,
                    ylabel = ylabel,
                    plotType = plotType,
                    plotMode = plotMode,
                    xlabel = xlabel
                )
            
            # Save variable with number of traces (to delete them all before plotting other series)    
            self.ntraces = len(elids)
        
    # Define response function
    def response(self, change):
        
        stage_sub, type_sub, tperiod, elids = None, None, None, None
        plot = False

        if change['name'] == 'selected_index':
            
            selection = change['new']
            
            # Determine what type of data are being selected
            # Update the dclass, dtype and vtype variables accordingly
            if selection != None:
                
                titles = change['owner']._titles
                desc = titles[str(selection)]
                objNames = [item[1] for item in list(titles.items())]
                
                if objNames == self.dclasses:
                    
                    self.dclass = desc
                    # Load previous data type and variable type state
                    self.dtype = self.UIState[self.dclass][0]
                    self.vtype = self.UIState[self.dclass][1][self.dtype][0]
                    plot = True
                    
                elif objNames in [self.ltypes, self.hmi_types, self.val_types]:
                    
                    self.dtype = desc
                    # Update data type state for given data class
                    self.UIState[self.dclass][0] = desc
                    # Load previous variable type state for given data class
                    self.vtype = self.UIState[self.dclass][1][self.dtype][0]
                    plot = True
                    
                elif objNames in [['Stage'],['Type','Stage'],['Time Period','Element ID']]:
                    
                    self.vtype = desc  
                    self.UIState[self.dclass][1][self.dtype][0] = desc
        
        # The change is a data selection
        elif change['name'] == 'value':
            
            plot = True
            desc = change['owner'].description
            
            # The change is from a checkbox
            if desc:
                
                if change['new']: 
                    
                    self.UIState[self.dclass][1][self.dtype][1][self.vtype].append(desc)
                    
                else:
                    
                    self.UIState[self.dclass][1][self.dtype][1][self.vtype].remove(desc)
            
            # Radio buttons store the description of the change in change['new']
            if not desc:
                
                desc = change['new']
                self.UIState[self.dclass][1][self.dtype][1][self.vtype] = desc
        
        if plot:
            
            if self.dclass == 'Lab Results':
                    
                # Load previous state for lab results
                self.stage_sub = self.UIState['Lab Results'][1][self.dtype][1]['Stage']
                self.type_sub = self.UIState['Lab Results'][1][self.dtype][1]['Type']
                self.get_lab_plots(self.dtype, self.type_sub, self.stage_sub)

            elif self.dclass == 'HMI Data':

                # Load previous state for HMI data
                self.tperiod = self.UIState['HMI Data'][1][self.dtype][1]['Time Period']
                self.elids = self.UIState['HMI Data'][1][self.dtype][1]['Element ID']
                self.get_hmi_plots(self.dtype, self.elids, self.tperiod)
                


In [137]:
# Load CR2C Plots Class
cp = cr2cPlots()
# Create tab nests 
cp.create_tabNests()
# Display graph widget
display(gw)

Button(description='Select All', style=ButtonStyle())

Button(description='Clear All', style=ButtonStyle())

GraphWidget()